In [1]:
!pip install sentence-transformers

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import pickle
from scipy import spatial
import os
print(os.getcwd())

C:\Users\yiehy\OneDrive\Desktop\cs425-nlc-project\4.Retrieval\finetuned_bertbase


# Install/import relevant libraries

# Get embeddings for distilbert

In [2]:
qna_df = pd.read_csv("../../0.Datasets/QnA.csv")

In [3]:
qna_df.head()

,Unnamed: 0,qid,docid,question,answer
0,0,0,18850,What is considered a business expense on a bus...,The IRS Guidance pertaining to the subject. I...
1,1,1,14255,Claiming business expenses for a business with...,Yes you can claim your business deductions if ...
2,2,2,308938,Transferring money from One business checking ...,You should have separate files for each of the...
3,3,3,296717,Having a separate bank account for business/in...,Having a separate checking account for the bus...
4,4,3,100764,Having a separate bank account for business/in...,"You don't specify which country you are in, so..."


In [4]:
qna_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17072 entries, 0 to 17071
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  17072 non-null  int64 
 1   qid         17072 non-null  int64 
 2   docid       17072 non-null  int64 
 3   question    17072 non-null  object
 4   answer      17072 non-null  object
dtypes: int64(3), object(2)
memory usage: 667.0+ KB


In [5]:
model = SentenceTransformer("../../10.Fine-tuned Models/finetuned-bertbase-1epoch")

C:\Users\yiehy\anaconda3\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [11]:
answers_list = qna_df["answer"].tolist()

In [46]:
# split answers to chunks for model to encode
answers_list = qna_df["answer"].tolist()
chunk_size = 10
chunks = [answers_list[i:(i+chunk_size)] for i in range(0, len(answers_list), chunk_size)]

In [23]:
dimension_size = 768 # embedding dimensions from msmarco bertbase documentation
sentence_embeddings = np.empty([0,dimension_size], dtype=np.float32)
for i,chunk in enumerate(chunks):
    if i% chunk_size == 0:
        print(i* chunk_size)
    # stack it to sentence_embeddings numpy array
    sentence_embeddings = np.vstack((sentence_embeddings, model.encode(chunk)))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000


In [24]:
len(sentence_embeddings)

17072

In [25]:
#save model embeddings
with open("./answer_embeddings.pkl", 'wb') as f:
    pickle.dump(sentence_embeddings, f, pickle.HIGHEST_PROTOCOL)

# Show top 50 results for a query (for qualitative evaluation and re-ranking)

In [12]:
# Read embeddings
sentence_embeddings = pickle.load(open("./answer_embeddings.pkl", 'rb'))

In [13]:
test_query = "Should i use a robo advisor or hire a financial advisor?"

In [14]:
%%time
question_embedding = model.encode(test_query)
answer_similiarity = {}
for i,embed in enumerate(sentence_embeddings):
    answer_similiarity[i]= np.dot(question_embedding, embed)
answer_similiarity = {k: v for k, v in sorted(answer_similiarity.items(), key=lambda item: item[1], reverse=True)}
top_20_index = list(answer_similiarity)[:50]
for i,index in enumerate(top_50_index):
    print(f"========== Rank {i+1} ==========")
    print(answers_list[index])
    print()

========== Rank 1 ==========
If your financial needs aren't complex, and mostly limited to portfolio management, consider looking into the newish thing called robo-advisers (proper term is "Automated investing services").  The difference is that robo-advisers use software to manage portfolios on a large scale, generating big economy of scale and therefore offering a much cheaper services than personal advisor would - and unless your financial needs are extremely complex, the state of the art of scaled up portfolio management is at the point that a human advisor really doesn't give you any value-add (and - as other answers noted - human advisor can easily bring in downsides such as conflict of interest and lack of fiduciary responsibility).  disclaimer: I indirectly derive my living from a company which derives a very small part of their income from a robo-adviser, therefore there's a possible small conflict of interest in my answer

========== Rank 2 ==========
If you are looking for a

# Evaluate results on test dataset

In [6]:
test_df = pd.read_csv("../../0.Datasets/train_test_split/test.csv")

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3415 entries, 0 to 3414
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  3415 non-null   int64 
 1   qid         3415 non-null   int64 
 2   docid       3415 non-null   int64 
 3   question    3415 non-null   object
 4   answer      3415 non-null   object
dtypes: int64(3), object(2)
memory usage: 133.5+ KB


In [8]:
test_df.head().drop(columns=["Unnamed: 0"])

,qid,docid,question,answer
0,7817,31330,Can you have a positive return with a balance ...,Have you owned the stock for longer than 2015?...
1,6304,105557,Oversimplify it for me: the correct order of i...,Great questions -- the fact that you're thinki...
2,7115,43508,Definition of “U.S. source” for US non-residen...,The examples you provide in the question are c...
3,5716,287327,Are car buying services worth it?,I have used car buying services through Costco...
4,9016,580920,What makes a Company's Stock prices go up or d...,Here are some significant factors affect the c...


In [26]:
# split answers to chunks for model to encode
test_answers_list = test_df["answer"].tolist()
chunk_size = 10
chunks = [test_answers_list[i:(i+chunk_size)] for i in range(0, len(test_answers_list), chunk_size)]
dimension_size = 768 # embedding dimensions from msmarco bertbase documentation

test_answer_embeddings = np.empty([0,dimension_size], dtype=np.float32)
for i,chunk in enumerate(chunks):
    if i% chunk_size == 0:
        print(i* chunk_size)
    # stack it to test_answer_embeddings numpy array
    test_answer_embeddings = np.vstack((test_answer_embeddings, model.encode(chunk)))
    
#save model embeddings
with open("./finetuned_bertbase_test_answer_embeddings.pkl", 'wb') as f:
    pickle.dump(test_answer_embeddings, f, pickle.HIGHEST_PROTOCOL)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400


In [28]:
question_answer_index_map = {}
for _,row in test_df.iterrows():
    if row["qid"] not in question_answer_index_map:
        question_answer_index_map[row["qid"]]= []
        question_answer_index_map[row["qid"]].append(row["docid"])
    else:
        question_answer_index_map[row["qid"]].append(row["docid"])

In [29]:
labels = []
for v in question_answer_index_map.values():
    labels.append(v)

In [30]:
question_map = {}
label_map = {}
for _,row in test_df.iterrows():
    if row["qid"] not in question_map:
        question_map[row["qid"]] = row["question"]
    if row["answer"] not in label_map:
        label_map[row["answer"]] = row["docid"]

In [31]:
sentence_embeddings = pickle.load(open("./finetuned_bertbase_test_answer_embeddings.pkl", 'rb'))
test_answer_list = test_df["answer"].tolist()
predictions = []
count=1
for k,v in question_map.items():
    if count%100==0:
        print(count)
    question_embedding = model.encode(v)
    answer_similiarity = {}
    for i,embed in enumerate(sentence_embeddings):
        answer_similiarity[i]= np.dot(question_embedding, embed)
    answer_similiarity = {k: v for k, v in sorted(answer_similiarity.items(), key=lambda item: item[1], reverse=True)}
    response = list(answer_similiarity)[0]
    label_index = label_map[test_answer_list[response]]
    predictions.append([label_index])
    count+=1

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500


In [32]:
#save prediction and results
results = {"labels":labels,"predictions":predictions}
with open("../../7.Evaluate/finetuned_bertbase_results.pkl", 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)